In [10]:
!pip install openai
!pip install pandas
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [11]:
# Importing necessary libraries
import json
import openai
import os
import time
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_KEY")
data_fld="FourSchools"
swt_file_name="novel.txt"
task_file_name=""
setting_file_name="setting.md"

# テキストファイルからデータを読み込む関数
def load_text_from_file(file_name):
    with open(file_name, 'r',encoding='utf-8') as f:
        text = f.read()
    return text

In [6]:
import json

# スライディングウィンドウテクニック
def sliding_window_split(file_name, window_size=350, step_size=200):
    text = load_text_from_file(file_name)
    chunks = [text[i:i+window_size] for i in range(0, len(text)-window_size, step_size)]
    chat_data = []
    for i, chunk in enumerate(chunks[:-1]):
        system_info={'role': 'system', 'content': f'Enter the Friday Conclave 金曜会へようこそ:File: {file_name}, Chunk Index: {i}'}
        user_content = {'role': 'user', 'content': chunk}
        assistant_content = {'role': 'assistant', 'content': chunks[i+1]}
        chat_data.extend([system_info, user_content, assistant_content])

    return chat_data

sliding_window_data = sliding_window_split(data_fld+'/'+swt_file_name)

# JSONファイルとして保存
with open(f'jsonData/dumped_{swt_file_name}.json', 'w') as f:
    json.dump(sliding_window_data, f)

In [7]:
import re
from collections import deque

def markdown_to_json(file_name):
    text=load_text_from_file(file_name)
    lines = text.strip().split('\n')
    json_array = []
    primary = ''
    secondary = deque()
    data_struct = 'Enter the Friday Conclave 金曜会へようこそ:Setting Data'
    midx = None
    is_primary=False

    for line in lines:
        if line.startswith('# '):
            primary = line[2:]
            json_array.append({'role': 'system', 'content': f'{data_struct} > {primary}'})
            is_primary=True
        elif line.startswith('## '):
            if secondary:
                json_array.extend(secondary)
            secondary.clear()
            secondary.append({'role': 'system', 'content': f'{primary}'})

            secondary_name = line[3:]
            secondary.append({'role': 'user', 'content': f'{secondary_name}'})
            midx = len(secondary)
        elif line.startswith('### '):
            sub_name = line[4:]
            secondary[midx]['content'] += f'&{sub_name}'
            secondary.append({'role': 'system', 'content': f'{data_struct} > {primary} > {secondary_name}'})
            secondary.append({'role': 'user', 'content': f'{secondary_name} > {sub_name}'})
        else:
            if is_primary:
                primary+=(':'+line)
                is_primary=False
            else:
                secondary.append({'role': 'assistant', 'content': line})

    if secondary:
        json_array.extend(secondary)

    return json_array
markdown_data = markdown_to_json(data_fld+'/'+setting_file_name)
with open(f'jsonData/dumped_{setting_file_name}.json', 'w') as f:
    json.dump(markdown_data, f)

In [7]:
import os
import json
from pprint import pprint

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            pprint(data)  # Add this line to print the data
            return data
        except json.JSONDecodeError as e:
            return None

json_dir = './jsonData/'
json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]
json_merged=[]
for json_file in json_files:
    file_path = os.path.join(json_dir, json_file)
    data = read_json_file(file_path)
    if data is not None:
        json_merged.extend(data)
    # else block removed

messages={
     "messages":json_merged
}

with open(f'converted_data.jsonl', 'w') as f:
    json.dump(messages, f)
  # Add this line to print the final messages object


[{'content': 'Enter the Friday Conclave 金曜会へようこそ:File: FourSchools/novel.txt, '
             'Chunk Index: 0',
  'role': 'system'},
 {'content': 'プロローグ\n'
             '\n'
             '\u3000'
             '望楼への階段を駆け上がると四璧祭を破壊した業火の熱が俺を照らした。帝都の夜を赫く焦がす火災はすでに魔導院の大半を焼き尽くし街にまで溢れ出ようとしていた。人々の悲鳴や怒号、消火魔法の詠唱や延焼を防ぐ為に壊れかけた建物を撃つ砲撃の音がその地獄に華を添える。\n'
             '\n'
             '\u3000見回すと予想通りそこにはヒビ割れた仮面を被った怪盗イコノクラスタが静かに立っていた。\n'
             '\n'
             '「…これがあなたの望みなんですか？」\n'
             '\n'
             '「悪い物語は壊され無ければならない」\n'
             '\n'
             '「俺には悪い物語の続きに見えるけど…何処にも回収されない。ただ無意味に人が死んで行くだけ。違うんですか？」\n'
             '\n'
             '「意味が有れば死んで良いのか？…違う！ただそんな意味の為に死んで欲しく無かっただけなんだ！だから僕はやる」\n'
             '\n'
             '\u3000激した仮面の人物は腕を振り上げ手に持',
  'role': 'user'},
 {'content': '\n'
             '「悪い物語は壊され無ければならない」\n'
             '\n'
             '「俺には悪い物語の続きに見えるけど…何処にも回収されない。ただ無意味に人が死んで行くだけ。違うんですか？」\n'
             '\n'
             '「意味が有れば死んで良いのか？…違う！ただそんな意味の為に死んで欲しく無かっ

In [15]:
import os
import time

# Upload the formatted data file
file = openai.File.create(
  file=open("converted_data.jsonl", "rb"),
  purpose='fine-tune'
)

# Check file status
counter=0
file_status = openai.File.retrieve(file['id'])
while file_status['status'] != 'processed':
  print(f'Waiting for file to be ready... {file_status["status"]}')
  if counter%10==0:
        print(file_status)
 
  time.sleep(5)
  file_status = openai.File.retrieve(file['id'])
  counter+=1



Waiting for file to be ready... uploaded
{
  "object": "file",
  "id": "file-N1flvmaKOTrbC9D4xApAcSkM",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1768163,
  "created_at": 1693209690,
  "status": "uploaded",
  "status_details": null
}
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
{
  "object": "file",
  "id": "file-N1flvmaKOTrbC9D4xApAcSkM",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1768163,
  "created_at": 1693209690,
  "status": "uploaded",
  "status_details": null
}
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waiting for file to be ready... uploaded
Waitin

In [17]:
with open("uploaded_file.json","w") as f:
    json.dump(file_status,f)
    

In [19]:
with open("uploaded_file.json","r",encoding="utf-8") as f:
    file=json.load(f)

# Create a fine-tuning job
fine_tuning_job = openai.FineTuningJob.create(
  training_file=file['id'],
  model="gpt-3.5-turbo"
)
conter=0
tuning_status=openai.FineTuningJob.retrieve(fine_tuning_job['id'])
while tuning_status['status'] != 'succeeded':
  print(f'Waiting for tuning to succeeded... {tuning_status["status"]}')
  if counter%10==0:
        print(tuning_status)
 
  time.sleep(5)
  tuning_status = openai.FineTuningJob.retrieve(fine_tuning_job['id'])
  counter+=1

# Print the fine-tuning job details


with open('fine_tuning_results.json', 'w') as json_file:
  json.dump(tuning_staus, json_file)
tuning_status

InvalidRequestError: file-N1flvmaKOTrbC9D4xApAcSkM has 1 example(s), but must have at least 10 examples

In [ ]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve("ft-abc123")

# Cancel a job
openai.FineTuningJob.cancel("ft-abc123")

# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="ft-abc123", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
import openai
openai.Model.delete("ft-abc123")

In [14]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x248c9654450> JSON: {
  "object": "list",
  "data": [],
  "has_more": false
}

In [ ]:
openai.FineTuningJob.cancel("ft-abc123")